In [ ]:
 #masukan modul yang dibutuhkan
import pandas as pd
import numpy as np
import nltk
import string
import re

In [ ]:
#masukan data kembali yang akan diproses
def load_data():
    data_content = pd.read_csv('threads.csv')
    return data_content

df = load_data()
df.head()

In [ ]:
#cleaning
def cleaning(content):
    content = re.sub(r'\$\w*', '', str(content))         #digunakan untuk menghapus semua kata yang dimulai dengan tanda dolar ($) dan diikuti oleh karakter huruf, angka, atau garis bawah.      #
    content = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', content)     #untuk menghapus semua URL atau tautan web dari teks.
    content = re.sub('&quot;'," ", content)         #Digunakan untuk menggantikan setiap kemunculan `&quot;` dengan spasi kosong dalam kolom `text`.
    content = re.sub(r"\d+", " ", str(content))         #digunakan untuk menggantikan semua angka dalam teks yang disimpan dalam kolom `text` dengan spasi kosong.
    content = re.sub(r"\b[a-zA-Z]\b", "", str(content))     #digunakan untuk menghapus semua kata tunggal dalam teks yang disimpan dalam kolom `text`.
    content = re.sub(r"[^\w\s]", " ", str(content))         #digunakan untuk menggantikan semua karakter non-alphanumerik dan non-spasi dalam teks yang disimpan dalam variabel `content` dengan spasi kosong.
    content = re.sub(r'(.)\1+', r'\1\1', content)         #Digunakan untuk mengganti dua atau lebih karakter berulang dalam teks dengan hanya dua karakter yang berulang. Misalnya, jika terdapat karakter berulang "eeeee" dalam teks, maka akan digantikan dengan "ee".
    content = re.sub(r"\s+", " ", str(content))       #digunakan untuk menggantikan satu atau lebih spasi berturut-turut dalam teks
    content = re.sub(r'#', '', content)         #digunakan untuk menghapus semua tanda pagar (#) dalam teks
    content = re.sub(r'[^a-zA-z0-9]', ' ', str(content))    #Digunakan untuk menggantikan semua karakter non-alphanumerik dalam teks dengan spasi kosong, sehingga menghapus karakter-karakter tersebut dari teks dan mempertahankan hanya huruf (kapital dan kecil) serta angka.
    content = re.sub(r'\b\w{1,2}\b', '', content)     #digunakan untuk menghapus kata-kata dengan panjang satu atau dua karakter dalam teks
    content = re.sub(r'\s\s+', ' ', content)      #Digunakan untuk menggantikan dua atau lebih spasi berturut-turut dalam teks dengan satu spasi tunggal.
    content = re.sub(r'^RT[\s]+', '', content)        #menghapus RT
    content = re.sub(r'^b[\s]+', '', content)       #digunakan untuk menghapus spasi di awal teks
    content = re.sub(r'^link[\s]+', '', content)      #digunakan untuk menghapus string "link" yang diikuti oleh spasi di awal teks
    return content

def remove_emoji(content):
    emoji = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # simbol & piktogram
                        u"\U0001F680-\U0001F6FF"  # transportasi & simbol peralatan
                        u"\U0001F1E0-\U0001F1FF"  # bendera negara
                        u"\U00002702-\U000027B0"  # simbol
                        u"\U000024C2-\U0001F251"  # emoji lainnya
                        "]+", flags=re.UNICODE)
    return emoji.sub(r'', content)

df['cleaning'] = df['content'].apply(cleaning)
df.head()

In [ ]:
#case folding - ubah jadi huruf kecil
df['case_folding'] = df['cleaning'].str.lower()
df.head()

In [ ]:
#tokenisasi - membagi kalimat jadi perkata (dipisah)
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def word_tokenize_wrapper(content):
    return word_tokenize(content)

df['tokenisasi'] = df['case_folding'].apply(lambda x: word_tokenize_wrapper(x.lower()))
df.head()

In [ ]:
#Normalisasi-menormalisasikan kata yang non formal menjadi formal sesuai dengan kamus colloquial-indonesian-lexicon
def normalization (content):
  content_slang = pd.read_csv('colloquial-indonesian-lexicon.csv')
  dict_slang ={}
  for i in range(content_slang.shape[0]):
    dict_slang[content_slang["slang"][i]]=content_slang["formal"][i]

  drop_slang = []
  for teks in content:
    normalisasi_teks = [dict_slang[word] if word in dict_slang.keys() else word for word in teks]
    drop_slang.append(normalisasi_teks)

  return drop_slang

df['normalisasi'] = normalization(df['tokenisasi'])
df.head()

In [ ]:
#stopword removal - menghapus kata sesuai dengan kamus indonesia
nltk.download('stopwords')
from nltk.corpus import stopwords

list_stopwords = stopwords.words('indonesian')

#tambahan kata
list_stopwords.extend(['yg', 'dg', 'rt', 'dgn', 'ny', 'gt', 'klo',
                       'kalo', 'amp', 'biar', 'xad', 'xef',
                       'gak', 'xbc', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'xae', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&', 'yah', 'no', 'je', 'xbb', 'xb', 'sch',
                       'injirrr', 'ah', 'oena', 'bu', 'eh', 'xac', 'anjir'])

list_stopwords = set(list_stopwords)

def stopwords_removal(content):
  return [word for word in content if word not in list_stopwords]

df['stopword_removal'] = df['normalisasi'].apply(stopwords_removal)
df.head()

In [ ]:
#stemming - menghapus imbuhan
!pip install swifter
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import swifter

#buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#stemmed wrapper
def stemmed_wrapper(term):
  return stemmer.stem(term)

term_dict = {}

for content in df['stopword_removal']:
    for term in content:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")

#memmulai stemming
def apply_stemmed_term(full_text):
  return [term_dict[term] for term in full_text]

df['stemming'] = df['stopword_removal'].swifter.apply(apply_stemmed_term)
df.head()

In [ ]:
#buat stemming bebas dari kurung siku
stemming = df[['stemming']]

def fit_stemming(content):
    content = np.array(content)
    content = ' '.join(content)

    return content

df['stemming'] = df['stemming'].apply(lambda x: fit_stemming(x))
df.head()

In [ ]:
#remove kalimat duplikat dari kolom stemming
df.drop_duplicates(subset = "stemming", keep = 'first', inplace = True)
df

In [ ]:
#simpan kedalam csv
df.to_csv('preprocessing.csv', sep=',', index=False)